In [1]:
!pip install spacy > /dev/null
!python -m spacy download nb_core_news_sm > /dev/null

In [2]:
import pandas as pd
import spacy
from spacy.lang.nb.examples import sentences 
from nltk.tokenize import word_tokenize

In [3]:
def filter_split_dataset(metadata_df = None):
    """Filter out nynorsk, and split into train and test datasets.
    
    :param metadata_df: pd.DataFrame object with metadata.
    
    :return: 2-tuple of train and test pd.DataFrames
    """
    if metadata_df is None:
        metadata_df = pd.read_json(f"data/metadata.json").T
        
    bokmål_mask = metadata_df.language == "nb"
    
    metadata_df = metadata_df[bokmål_mask]
    
    train_mask = metadata_df.split == "train"
    test_mask = metadata_df.split == "test"
        
    return metadata_df[train_mask], metadata_df[test_mask]
    

In [4]:
def process_documents(docs, lemmatizer = None):
    """Tokenize and lemmatize.
    
    :param document: String.
    :param lemmatizer: Function Str -> iter(Str,)
    """
        
    if lemmatizer is None:
        lemmatizer = spacy.load("nb_core_news_sm")
    
    return lemmatizer(docs)

In [11]:
def get_documents(metadata_df=None, path="data", processed=False):
    """Get documents of a specific type.
    
    Filter out reviews in nynorsk.
    
    :param path: Str path to folder with metadata.json file, and test and train folders.
    :param dataset: Determines which type to look for. Either `train` or `test`.
    :param processed: Whether to look for already processed data or not.
    
    :return: iter(Str,) of documents
    """
    
    if metadata_df is None:
        metadata_df = pd.read_json(f"{path}/metadata.json").T
    
    full_path = f"{path}/%s/%s{'p' if processed else ''}.txt"
    
    for (_, review) in metadata_df.iterrows():
        document = open(f"{path}/{review['split']}/{str(review['id']).zfill(6)}{'p' if processed else ''}.txt", "r").read()
        yield document, review["rating"], review["authors"]

In [6]:
metadata_df = pd.read_json(f"data/metadata.json").T

In [8]:
display(metadata_df)

,authors,category,day,excerpt,id,language,month,rating,source,source-category,source-id,source-tags,split,tags,title,url,year,cons,pros
0,[Birger Vestmo],screen,27,Toppen innen tv-drama akkurat nå!,0,nb,9,6,p3,tv,74781,[],train,[tv],Rome S02,https://p3.no/filmpolitiet/category/tv/rome-02,2007,NaN,NaN
1,[Birger Vestmo],screen,22,Gull for <em>Twin Peaks</em>-fans!,1,nb,11,6,p3,tv,74065,[],train,[tv],Twin Peaks - definitive gold box edition,https://p3.no/filmpolitiet/category/tv/twin-pe...,2007,NaN,NaN
2,[Birger Vestmo],screen,22,"The Wire vil gjøre deg avhengig, men på en god...",2,nb,2,6,p3,tv,73886,[],train,[tv],The Wire (sesong 1-4),https://p3.no/filmpolitiet/category/tv/the-wir...,2008,NaN,NaN
3,[Birger Vestmo],screen,3,"Stilig, underholdende og sofistikert!",3,nb,12,5,p3,tv,77072,[],train,[tv],Mad Men (sesong 1),https://p3.no/filmpolitiet/category/tv/mad-men...,2008,NaN,NaN
4,[Birger Vestmo],screen,9,TV-underholdning av høyeste kvalitet!,4,nb,7,5,p3,film,34894,[],train,[movie],Mad Men (sesong 2),https://p3.no/filmpolitiet/category/tv/mad-men...,2009,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706991,[Anette Basso],misc,5,Gamle helter freser fra seg med fornyet styrke...,706991,nb,5,5,sa,kultur,70789K,"[Anmeldelse, Bokanmeldelse]",test,[],Politisk protestpønk fra gamle helter,https://aftenbladet.no/i/70789K/,2019,NaN,NaN
706992,[Geir Flatøe],misc,25,Ben Harper kommer fra en familie av Mavis Stap...,706992,nb,5,5,sa,kultur,vQlRaB,"[Anmeldelse, Plateanmeldelse]",test,[],79-åringen står fortsatt på barrikadene,https://aftenbladet.no/i/vQlRaB/,2019,NaN,NaN
706993,[Kine Hult],misc,16,«Ansiktet» utfordrer på en ubehagelig måte vår...,706993,nb,5,4,sa,kultur,lAJrL7,"[Anmeldelse, Filmanmeldelse]",test,[movie],Utseendet teller ikke - eller?,https://aftenbladet.no/i/lAJrL7/,2019,NaN,NaN
706994,[Steinar Sivertsen],misc,22,BOK: Forsert og frisinna parademarsj i stivbei...,706994,nb,5,3,sa,kultur,wPxpPn,"[Roman, Litteratur, Anmeldelse, Bokanmeldelse,...",test,[],Hektisk homsesex med tårefylte forviklingar,https://aftenbladet.no/i/wPxpPn/,2019,NaN,NaN


In [7]:
train_metadata_df, test_metadata_df = filter_split_dataset(metadata_df = metadata_df)
lemmatizer = spacy.load("nb_core_news_sm")

In [36]:
for i, (document, rating, authors) in enumerate(get_documents(metadata_df = test_metadata_df)):
    pass
    if i > 0:
        break
    #print(i)
    for token in process_documents(document):
        print(token.text, token.lemma_)

Outlander Outlander
S02 S02
E01 E01
- -
02 02

 

Fortsatt fortsatt
et et
eventyr eventyr
du du
blir bli
glad glad
i. i.


 


Tidsreiseeventyret Tidsreiseeventyret
Outlander Outlander
, ,
basert basere
på på
Diana Diana
Gabaldons Gabaldons
populære populær
bokserie bokserie
, ,
kommer komme
endelig endelig
tilbake tilbake
på på
TV-skjermen tv-skjerm
denne denne
uken uke
. .


 


Der de
forrige forrige
sesong sesong
foregikk foregå
i i
det det
skotske skotsk
høylandet høyland
på på
midten midt
av av
1700-tallet 1700-tall
, ,
går gåre
turen tur
denne denne
gangen gange
til til
Frankrike Frankrike
og og
Paris Paris
. .

 

Det det
er er
dermed dermed
duket duke
for for
politiske politisk
intriger intrige
, ,
overdådige overdådig
hoffball hoffball
og og
påkostede påkoste
kjoler kjole
– –
en en
forandring forandring
som som
fryder fryd
og og
engasjerer engasjere
. .


 


Serien Serien
holder holde
fortsatt fortsatt
på på
sin sin
lune lun
humor humor
, ,
og og
gjensynet gjensyn
med med
vå

In [37]:
metadata_df.num_authors = metadata_df.authors.apply(lambda authors: len(authors))
all_authors = []
for authors in metadata_df.authors:
    all_authors.extend(authors)
all_authors = list(set(all_authors))
len(all_authors)
#genders = []
#for author in all_authors:
#    gender = input("")

602